In [2]:
import os 
import copy 
import torch
import argparse
import numpy as np
import torch.nn as nn
from timm.models import create_model
from datasets import build_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset,TensorDataset
from torchvision.datasets.folder import ImageFolder

import models
from pruning_utils import *
from layers import Conv2d, Linear
from sp_vision_transformer import Attention, Mlp

ModuleNotFoundError: No module named 'timm'

In [ ]:
from timm.models import create_model, safe_model_name, resume_checkpoint, load_checkpoint, model_parameters
from timm.data import create_dataset, create_loader, resolve_data_config

In [ ]:
#手动定义参数类，专门存储训练的参数，
class cfg(object):
    pass
args = cfg()#实例化
args.epochs, args.learning_rate, args.patience = [10, 0.001, 4]
args.hidden_size, args.input_size= [40, 30]

args.model = 'vit_small_patch16_224'

args.dataset = 'torch/CIFAR10'
args.data_dir = '../datasets'

args.batch_size = 32
args.pin_mem = 10


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#strip/gmp/amp_50_small.sh
args.sparsity=0.4
args.atten_density=0.5
args.mlp_density=0.5
args.heads=12
args.pretrained='https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth'
args.save_file='deit_base_patch16_224_omp40_mag.pt'
args.data=None
args.batch_size=128
args.type='mag'
args.model='deit_base_patch16_224'
args.input_size=224
args.drop=0.0
args.drop_path=0.1

In [ ]:
args

In [ ]:
def check_atten(binary_tensor, dim=64):
    heads_num = binary_tensor.shape[0] // dim
    output_list = []
    for i in range(heads_num):
        activated = binary_tensor[i*dim:i*dim+dim].mean().byte().float()
        output_list.append(activated.item())
    return output_list

def prune_loop(model, loss, pruner, dataloader, device, sparsity, scope, epochs, train_mode=False):

    # Set model to train or eval mode
    model.train()
    if not train_mode:
        model.eval()

    # Prune model
    for epoch in range(epochs):
        pruner.score(model, loss, dataloader, device)
        sparse = sparsity**((epoch + 1) / epochs) # represent density actually.
        pruner.mask(sparse, scope)

def prune_conv_linear(model):

    for name, module in reversed(model._modules.items()):

        if len(list(module.children())) > 0:
            model._modules[name] = prune_conv_linear(model=module)

        if isinstance(module, nn.Linear):
            bias=True
            if module.bias == None:
                bias=False
            layer_new = Linear(module.in_features, module.out_features, bias)
            model._modules[name] = layer_new

        if isinstance(module, nn.Conv2d):
            layer_new = Conv2d(module.in_channels, module.out_channels, module.kernel_size, module.stride)
            model._modules[name] = layer_new

    return model

In [ ]:

if args.type == 'mag':

    print('#########################################')
    print('########## Magnitude ####################')
    print('#########################################')

    number_examples = 100
    data = torch.ones(number_examples, 3, args.input_size, args.input_size)
    target = torch.ones(number_examples)
    data_set = torch.utils.data.TensorDataset(data, target)
    loader = torch.utils.data.DataLoader(data_set, 
        batch_size=1, shuffle=False, 
        num_workers=2, pin_memory=True)
    
if args.type == 'mag':
    print(f"Creating model: {args.model}")
    model = create_model(
        args.model,
        pretrained=False,
        num_classes=1000,
        drop_rate=args.drop,
        drop_path_rate=args.drop_path,
        drop_block_rate=None,
    )
if args.type == 'mag':
    print('loading pretrained weight')
    checkpoint = torch.hub.load_state_dict_from_url(
        args.pretrained, map_location='cpu', check_hash=True)
    model.load_state_dict(checkpoint['model'],strict=False)#改了一下，不然提示Missing key(s) in state_dict: "gumbel.weight", "gumbel.bias". 

    save_state_dict = copy.deepcopy(model.state_dict())
if args.type == 'mag':    
    prune_conv_linear(model)#moedl._modules[name] = layer_new  创造新的conv fc

    for key in save_state_dict.keys():
        if not key in model.state_dict().keys():
            print('can not load key = {}'.format(key))

    model.load_state_dict(save_state_dict, strict=False)
    model.cuda()

if args.type == 'mag':
    pruner = Mag(masked_parameters(model))
    prune_loop(model, None, pruner, loader, torch.device('cuda:0'), args.sparsity, scope='global', epochs=1, train_mode=True)
    print('Density = {}'.format(args.sparsity))


    current_mask = extract_mask(model.state_dict())
    check_sparsity_dict(current_mask)

if args.type == 'mag':
    torch.save(current_mask, args.save_file)